# Testing with making the bubble plot

In [120]:
import pandas as pd
import matplotlib.image as mpimg
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import image, pyplot
import matplotlib.image as mpimg
import  PIL.Image as Image


In [168]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

class model_trainer():
    def __init__(self, name):
        self.car_type = "5"

        self.df = pd.read_csv("..\..\data\MC1\SensorDataProcessed.csv")

    def run_prediction(self):
        # read df from csv
        df_x = pd.read_csv("..\..\data\MC1\one_hot_encoding.csv")
        y_values = self.get_y_values(self.car_type)
        x_train, x_test, y_train, y_test = train_test_split(df_x, y_values, test_size=0.20, random_state=69)

        logistic_model = LogisticRegression(random_state=0).fit(x_train, y_train)
        # return the coefficients of the model
        return logistic_model.coef_[0]
    
    def get_y_values(self, car_type):
        
        def is_car_type(type):
            return "1" if type == car_type else "0"

        cars = self.df.groupby(by="car-id").first()
        encoded_cars = cars.replace({"car-type": {"1": is_car_type("1"),
                                                  "2": is_car_type("2"),
                                                  "2P": is_car_type("2P"),
                                                  "3": is_car_type("3"),
                                                  "4": is_car_type("4"),
                                                  "5": is_car_type("5"),
                                                  "6": is_car_type("6")}})

        return encoded_cars.sort_values(by="Timestamp")['car-type']

In [169]:
trainer = model_trainer('test')
coefficients = trainer.run_prediction()

In [170]:
coefficients

array([ 0.67129476,  0.14518929,  0.14341058,  0.08086764,  0.08087018,
       -0.73168287,  1.11733022,  2.06220391, -1.09740183, -0.40748828,
        0.2965917 , -3.12960688, -2.74970358, -2.76504049, -0.52632552,
       -0.52632552, -2.4882633 , -0.24832654, -0.24832654, -3.42138793,
       -0.59318046, -1.80116245, -0.09834901, -1.78391021, -0.09834901,
       -0.31158982, -0.31158982,  0.69480982, -2.98636549, -0.11732294,
       -0.27656736, -0.13895315, -0.13895315, -2.89677379, -0.01818731,
       -0.47940568, -1.09409689, -1.27672534, -0.00956629, -0.00956629])

In [171]:
locations = pd.read_parquet("..\..\data\MC1\locations.parquet")
coordinates = locations['coordinates']
img = mpimg.imread("..\..\data\MC1\Lekagul Roadways.bmp")
image = Image.open("..\..\data\MC1\Lekagul Roadways.bmp")



In [172]:
def make_car_bubbles(coordinates, coefficients, img)->go.Figure:
    # get width and height of image PIL
    img_width, img_height = image.size
    scale_factor = 3

    coefficients = [abs(x) for x in coefficients]

    x_values = [(x[0])*scale_factor for x in coordinates]
    y_values = [(x[1])*scale_factor for x in coordinates]

    df_plot = pd.DataFrame({'x': x_values, 'y': y_values, 'size': coefficients})

    fig = px.scatter(df_plot, x='x', y='y', size='size')

    fig.update_xaxes(
        visible=False,
    )

    fig.update_yaxes(
        visible=False,
    )

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=img)
    )

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    return fig

In [173]:
fig = make_car_bubbles(coordinates, coefficients, image)

In [174]:
fig.show()